ALGORITMO PREDICCIÓN

Primero se cargaran las librerías necesarias

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import randint, uniform
import plotly.graph_objects as go

A continuación se carga el ficheo desde GITHUB SalesOrder.csv

In [2]:
# Cargar el dataset
url = 'https://raw.githubusercontent.com/annaalfaro/TFM/main/SalesOrder.csv'
Sales_order = pd.read_csv(url)


En el siguiente código se convertián los formatos de los campos en fecha, se obtendrán las características de fecha en Año, mes, día y día de la semana. 
Se copnvertiran los códigos en nuevos códigos númericos para poder trabajarlos.
También se generar las "lag Features" són columnas añadidas:
lag_1 -> sería la cantidad definida para el día anterior
lag_7 -> cantidad definida para la semana anterior
lag_30 -> cantidda definida para el mes anterior
Creación de las "rolling Featuures): también son columnas añadidas
rolling_7 -> genera la media de las cantidades durante una semana
Rolling_30 -> genera la media de las cantidades durante un mes


In [3]:
# Convertir las columnas de fechas a formato datetime
Sales_order['SO_Date'] = pd.to_datetime(Sales_order['SO_Date'], errors='coerce')
Sales_order = Sales_order[['SO_Date', 'SO_CustomerItemid', 'SO_Quantity']]

# Crear características adicionales a partir de la fecha
Sales_order['Year'] = Sales_order['SO_Date'].dt.year
Sales_order['Month'] = Sales_order['SO_Date'].dt.month
Sales_order['DayOfYear'] = Sales_order['SO_Date'].dt.dayofyear
Sales_order['DayOfWeek'] = Sales_order['SO_Date'].dt.dayofweek

# Asegurar que SO_CustomerItemid sea numérico (convertir a códigos)
Sales_order['SO_CustomerItemid'] = Sales_order['SO_CustomerItemid'].astype('category').cat.codes

# Crear "lag features"
Sales_order['Lag_1'] = Sales_order['SO_Quantity'].shift(1)
Sales_order['Lag_7'] = Sales_order['SO_Quantity'].shift(7)
Sales_order['Lag_30'] = Sales_order['SO_Quantity'].shift(30)

# Crear rolling features (promedios móviles de demanda)
Sales_order['Rolling_7'] = Sales_order['SO_Quantity'].rolling(window=7).mean()
Sales_order['Rolling_30'] = Sales_order['SO_Quantity'].rolling(window=30).mean()

# Eliminar filas con valores nulos que se introducen al crear las lag features
Sales_order = Sales_order.dropna()


Se definen las variables X e Y y los datos de la base de datos se dividen en datos de entrenamiento y de test.

In [4]:
# Definir las características (X) y la variable objetivo (y)
X = Sales_order[['SO_CustomerItemid', 'Year', 'Month', 'DayOfYear', 'DayOfWeek', 'Lag_1', 'Lag_7', 'Lag_30', 'Rolling_7', 'Rolling_30']]
y = Sales_order['SO_Quantity']

# Dividir los datos en conjunto de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Modelo de regresión XGBoost

In [6]:
# Optimización de hiperparámetros para XGBoost
param_dist_xgb = {
    'n_estimators': randint(50, 300),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.2),
    'colsample_bytree': uniform(0.6, 0.4)
}
xgb_model = XGBRegressor()
#random_search_xgb = RandomizedSearchCV(xgb_model, param_distributions=param_dist_xgb, n_iter=20, scoring='neg_mean_squared_error', cv=3, random_state=42, n_jobs=-1)
random_search_xgb = RandomizedSearchCV(xgb_model, param_distributions=param_dist_xgb, n_iter=20, scoring='neg_mean_squared_error', cv=3, random_state=42, n_jobs=1)

random_search_xgb.fit(X_train, y_train)
best_xgb = random_search_xgb.best_estimator_


Modelo LightGBM

In [8]:
# Optimización de hiperparámetros para LightGBM
param_dist_lgb = {
    'n_estimators': randint(50, 300),
    'num_leaves': randint(20, 150),
    'learning_rate': uniform(0.01, 0.2),
    'feature_fraction': uniform(0.6, 0.4),
    'bagging_fraction': uniform(0.6, 0.4)
}
lgb_model = LGBMRegressor()
#random_search_lgb = RandomizedSearchCV(lgb_model, param_distributions=param_dist_lgb, n_iter=20, scoring='neg_mean_squared_error', cv=3, random_state=42, n_jobs=-1)
random_search_lgb = RandomizedSearchCV(lgb_model, param_distributions=param_dist_lgb, n_iter=20, scoring='neg_mean_squared_error', cv=3, random_state=42, n_jobs=1)

random_search_lgb.fit(X_train, y_train)
best_lgb = random_search_lgb.best_estimator_

[LightGBM] [Warning] bagging_fraction is set=0.749816047538945, subsample=1.0 will be ignored. Current value: bagging_fraction=0.749816047538945
[LightGBM] [Warning] feature_fraction is set=0.9802857225639665, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9802857225639665
[LightGBM] [Warning] bagging_fraction is set=0.749816047538945, subsample=1.0 will be ignored. Current value: bagging_fraction=0.749816047538945
[LightGBM] [Warning] feature_fraction is set=0.9802857225639665, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9802857225639665
[LightGBM] [Warning] bagging_fraction is set=0.6624074561769746, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6624074561769746
[LightGBM] [Warning] feature_fraction is set=0.662397808134481, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.662397808134481
[LightGBM] [Warning] bagging_fraction is set=0.6624074561769746, subsample=1.0 will be ignored. Current

[LightGBM] [Warning] bagging_fraction is set=0.7700623497964979, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7700623497964979
[LightGBM] [Warning] feature_fraction is set=0.6831766651472755, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6831766651472755
[LightGBM] [Warning] bagging_fraction is set=0.7700623497964979, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7700623497964979
[LightGBM] [Warning] feature_fraction is set=0.6831766651472755, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6831766651472755
[LightGBM] [Warning] bagging_fraction is set=0.9757995766256756, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9757995766256756
[LightGBM] [Warning] feature_fraction is set=0.9579309401710595, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9579309401710595
[LightGBM] [Warning] bagging_fraction is set=0.9757995766256756, subsample=1.0 will be ignored. C

[LightGBM] [Warning] bagging_fraction is set=0.9329770563201687, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9329770563201687
[LightGBM] [Warning] feature_fraction is set=0.6849356442713105, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6849356442713105


Modelo CatBoost

In [9]:
# Entrenar CatBoost
cat_model = CatBoostRegressor(iterations=500, learning_rate=0.1, depth=6, silent=True)
cat_model.fit(X_train, y_train)

Meta-modelo ElasticNet

In [10]:
# Definir los modelos base para el stacking
estimators = [
    ('xgb', best_xgb),
    ('lgbm', best_lgb),
    ('cat', cat_model)
]

# Meta-modelo ElasticNet para el stacking
stacking_model = StackingRegressor(estimators=estimators, final_estimator=ElasticNet(alpha=0.1, l1_ratio=0.7))
stacking_model.fit(X_train, y_train)


[LightGBM] [Warning] bagging_fraction is set=0.9329770563201687, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9329770563201687
[LightGBM] [Warning] feature_fraction is set=0.6849356442713105, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6849356442713105
[LightGBM] [Warning] bagging_fraction is set=0.9329770563201687, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9329770563201687
[LightGBM] [Warning] feature_fraction is set=0.6849356442713105, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6849356442713105
[LightGBM] [Warning] bagging_fraction is set=0.9329770563201687, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9329770563201687
[LightGBM] [Warning] feature_fraction is set=0.6849356442713105, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6849356442713105
[LightGBM] [Warning] bagging_fraction is set=0.9329770563201687, subsample=1.0 will be ignored. C

StackingRegressor(estimators=[('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.8832290311184181,
                                            device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=No...
                                            multi_strategy=None,
                                            n_estimators=137, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                              ('lgbm',
                               LGBMRegressor(bagging_fraction=0.9329770563201687,
                                             feature_fraction=0.6849356442713105,
                                             learning_rate=0.04636499344142013,
                                             n_estimators=70, num_leaves=77)),
                              ('cat',
                               <catboost.core.CatBoostRegressor object at 0x164e32d90>)],
                  final_estimator=ElasticNet(alpha=0.1, l1_ratio=0.7))

Predicciones de las demandas futuras desde hoy a un año vista.
Generar un rango de fechas, se define una estimación entre 15-60 días despues de la fecha de recepción del la órden. Se calculan con los últimos 30 días las varables lag y rolling (retardos y promedios moviles), estas se añaden al conjunto de datos. PAra la previsión se utiliza un modelo stacking. 

In [15]:
# Predecir demandas futuras (desde hoy hasta un año después)
today = datetime.now()
futuro_df = pd.DataFrame()

# Generar un rango de fechas futuras (un año desde hoy)
futuro_df['SO_Date'] = pd.date_range(start=today, periods=365, freq='D')

# Generar la fecha estimada de entrega para cada SO (entre 15 y 60 días después de SO_Date)
futuro_df['SO_EstDate'] = futuro_df['SO_Date'] + pd.to_timedelta(np.random.randint(15, 61, size=len(futuro_df)), unit='D')

# Asignar CustomerItemid a las fechas futuras
futuro_df['SO_CustomerItemid'] = np.random.choice(data['SO_CustomerItemid'].unique(), len(futuro_df))

# Crear las mismas características para el conjunto de datos futuro
futuro_df['Year'] = futuro_df['SO_Date'].dt.year
futuro_df['Month'] = futuro_df['SO_Date'].dt.month
futuro_df['DayOfYear'] = futuro_df['SO_Date'].dt.dayofyear
futuro_df['DayOfWeek'] = futuro_df['SO_Date'].dt.dayofweek

# Usar los últimos datos de demanda histórica para crear las lag y rolling features
ultimos_datos = data.tail(30)  # Tomamos los últimos 30 días de datos históricos

# Calcular las lag features para las fechas futuras
for i in range(len(futuro_df)):
    lag_1 = ultimos_datos.iloc[-1]['SO_Quantity'] if not ultimos_datos.empty else 0
    lag_7 = ultimos_datos.iloc[-7]['SO_Quantity'] if len(ultimos_datos) >= 7 else 0
    lag_30 = ultimos_datos.iloc[-30]['SO_Quantity'] if len(ultimos_datos) >= 30 else 0
    rolling_7 = ultimos_datos['SO_Quantity'].rolling(7).mean().iloc[-1] if len(ultimos_datos) >= 7 else 0
    rolling_30 = ultimos_datos['SO_Quantity'].rolling(30).mean().iloc[-1] if len(ultimos_datos) >= 30 else 0

    futuro_df.loc[i, 'Lag_1'] = lag_1
    futuro_df.loc[i, 'Lag_7'] = lag_7
    futuro_df.loc[i, 'Lag_30'] = lag_30
    futuro_df.loc[i, 'Rolling_7'] = rolling_7
    futuro_df.loc[i, 'Rolling_30'] = rolling_30

    # Actualizar los datos históricos con la predicción para el siguiente día
    ultimos_datos = pd.concat([ultimos_datos, pd.DataFrame({'SO_Quantity': [lag_1]})], ignore_index=True)

# Seleccionar las columnas para la predicción
X_futuro = futuro_df[['SO_CustomerItemid', 'Year', 'Month', 'DayOfYear', 'DayOfWeek', 'Lag_1', 'Lag_7', 'Lag_30', 'Rolling_7', 'Rolling_30']]

# Hacer predicciones de la cantidad solicitada para el año futuro
predicciones_futuras = stacking_model.predict(X_futuro)

# Añadir las predicciones al DataFrame futuro
futuro_df['Predicted_Quantity'] = predicciones_futuras

# Ordenar las predicciones por fecha antes de guardarlas
futuro_df = futuro_df.sort_values(by='SO_Date')

# Guardar las predicciones futuras en un fichero CSV
futuro_df[['SO_Date', 'SO_EstDate', 'SO_CustomerItemid', 'Predicted_Quantity']].to_csv('prediccionesSO.csv', index=False)


Evaluación del modelo, usando:
- Error cuadratico medio MSE
- Raíz del error cuadrático medio RMSE
- Error absoluto medio MAE
- error porcentual absoluto medio MAPE
- Coeficiente de determinación R^2

In [16]:
# Evaluación de las métricas con el conjunto de prueba
y_pred_test = stacking_model.predict(X_test)

# Calcular las métricas
mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_test)
r2 = r2_score(y_test, y_pred_test)
mape = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100

print(f'Métricas de Validación en el Conjunto de Prueba:')
print(f'Error Cuadrático Medio (RMSE): {rmse:.2f}')
print(f'Error Absoluto Medio (MAE): {mae:.2f}')
print(f'Error porcentual absoluto medio (MAPE): {mape:.2f}%')
print(f'R-cuadrado (R²): {r2:.2f}')


Métricas de Validación en el Conjunto de Prueba:
Error Cuadrático Medio (RMSE): 6.56
Error Absoluto Medio (MAE): 5.38
Error porcentual absoluto medio (MAPE): 4.82%
R-cuadrado (R²): 0.75


A continuación se genera una gráfica con las cantidades originales y las pedichas, se ha añadido una marca conteniendo la tendencia media suavizada para no obtener picos muy desorbitados.

In [17]:

# Crear figura
fig = go.Figure()

# Ordenar los items únicos por su ID (para asegurar el orden en las gráficas)
items_ordenados = sorted(data['SO_CustomerItemid'].unique())

# Graficar demanda histórica y predicciones futuras, intercaladas por artículo
for item in items_ordenados:
    # Gráfico de la demanda histórica por artículo
    fig.add_trace(go.Scatter(x=data[data['SO_CustomerItemid'] == item]['SO_Date'],
                             y=data[data['SO_CustomerItemid'] == item]['SO_Quantity'],
                             mode='lines+markers', name=f'Histórico Item {item}',
                             line=dict(dash='solid')))
    
    # Gráfico de las predicciones futuras por artículo
    fig.add_trace(go.Scatter(x=futuro_df[futuro_df['SO_CustomerItemid'] == item]['SO_Date'],
                             y=futuro_df[futuro_df['SO_CustomerItemid'] == item]['Predicted_Quantity'],
                             mode='lines+markers', name=f'Predicción Item {item}',
                             line=dict(dash='dash')))

# Calcular la línea de tendencia general usando media móvil
# Concatenar los datos históricos y predicciones futuras
total_data = pd.concat([data[['SO_Date', 'SO_Quantity']], 
                        futuro_df[['SO_Date', 'Predicted_Quantity']].rename(columns={'Predicted_Quantity': 'SO_Quantity'})])

# Ordenar los datos por fecha para el cálculo de la media móvil
total_data = total_data.sort_values('SO_Date')

# Aplicar una media móvil de 7 días (ajustable) para suavizar los datos
media_movil = total_data['SO_Quantity'].rolling(window=7).mean()

# Añadir la línea de tendencia suavizada a la gráfica
fig.add_trace(go.Scatter(x=total_data['SO_Date'],
                         y=media_movil,
                         mode='lines', name='Tendencia Media Suavizada',
                         line=dict(color='firebrick', width=4, dash='dot')))

# Actualizar los ejes y el diseño de la gráfica
fig.update_layout(title='Demanda por Artículo (Histórica vs Predicha para el Futuro)',
                  xaxis_title='Fecha',
                  yaxis_title='Cantidad',
                  hovermode='x unified',
                  legend_title='Artículos')

# Mostrar la gráfica interactiva
fig.show()
